In [0]:
import pandas as pd
import numpy as np
import heapq
import io


np.random.seed(seed=7)
# Load data
# interactions = pd.read_csv('./data/interactions.csv', sep='\t')
# items = pd.read_csv('./data_modified/item_profile.csv', sep='\t')
# users = pd.read_csv('./data/user_profile.csv', sep='\t')
# interactions

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving item_profile.csv to item_profile (3).csv
User uploaded file "item_profile.csv" with length 21451151 bytes


In [0]:
users = pd.read_csv(io.StringIO(uploaded["user_profile.csv"].decode('utf-8')), delimiter='\t')

In [6]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving item_profile.csv to item_profile (2).csv
User uploaded file "item_profile.csv" with length 21451151 bytes


In [0]:
items = pd.read_csv(io.StringIO(uploaded["item_profile.csv"].decode('utf-8')), delimiter='\t')

In [8]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving interactions.csv to interactions (2).csv
User uploaded file "interactions.csv" with length 15444858 bytes


In [0]:
interactions = pd.read_csv(io.StringIO(uploaded["interactions.csv"].decode('utf-8')), delimiter='\t')

In [0]:
# TODO: Precalculate all user-item scores.
def get_interactions_score(user_id, item_id):
    user_items = interactions[(interactions['user_id'] == user_id) & (interactions['item_id'] == item_id)]
    score = 0
    for i, row in user_items.iterrows():
        score += row['interaction_type']
    return score

def get_role_title_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    
    job_roles = user['jobroles'].split(',')
    job_titles = item['title'].split(',')
    score = 0
    for role in job_roles:
        for title in job_titles:
            if title == role:
                score += 1
                
    return score

def get_career_level_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['career_level'] == item['career_level']


def get_discipline_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['discipline_id'] == item['discipline_id']

def get_industry_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['industry_id'] == item['industry_id']

def get_regions_match(user_id, item_id):
    user = users[users['user_id'] == user_id].iloc[0]
    item = items[items['id'] == item_id].iloc[0]
    return user['region'] == item['region']

W1 = 100
W2 = 10
W3 = 12
W4 = 10
W5 = 5
W6 = 2

def heuristic_score_for_item(user_id, item_id):
    return np.sum(np.asarray([W1, W2, W3, W4, W5, W6]) * np.asarray([
        get_interactions_score(user_id, item_id),
        get_role_title_match(user_id, item_id),
        get_career_level_match(user_id, item_id),
        get_discipline_match(user_id, item_id),
        get_industry_match(user_id, item_id),
        get_regions_match(user_id, item_id)
    ]))

# Works very slow, should precalculate scores.
def heuristic_score(user_id, rec_count=5):
    recommendations = []
    for _, row in items.iterrows():
        score = heuristic_score_for_item(user_id, row['id'])
        heapq.heappush(recommendations, score)
        
    top_scores = []
    for i in range(rec_count):
        top_scores.append(heapq.heappop(recommendations))
    return top_scores


# user = users[users['user_id'] == 1731019].iloc[0]
# item = items[items['id'] == 2397541].iloc[0]
# user_items = interactions[(interactions['interaction_type'] == 3)]

# print(user_items)
# print(item)
# print(user)

# print(heuristic_score_for_item(1731019, 2397541))

# print(heuristic_score(1286))


    

    

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

id_to_idx = {}
idx_to_item = {}

items = items.drop(['title', 'career_level', 'country', 'tags'], axis=1)

for i, item in items.iterrows():
        id_to_idx[item['id']] = i
        idx_to_item[i] = item




In [0]:
item_matrix = items.values
# item_matrix[item_matrix == ''] = '0'
# item_matrix = np.delete(items.values, [0, 1, 2, 5, 10], 1).astype(float)
# id	title	career_level	discipline_id	industry_id	country	region	latitude	longitude	employment	tags	created_at	active_during_test

In [0]:
def item_item_CF(items):
    print(np.shape(items))
    return cosine_similarity(items, items)    

In [0]:
from scipy.cluster.vq import kmeans2
import numpy as np
import math
import sklearn

# Clusterizition
def get_clusters(items_matrix, num_clusters=10):    
    return kmeans2(items_matrix, num_clusters)
    

# print(item_matrix[math.isnan(item_matrix)])
    

centroid, item_clusters = get_clusters(item_matrix)
# print('item clusts', item_clusters)
similarities = {}
local_indexing = {}

def get_item_cluster(item_id):
    idx = id_to_idx[item_id]
    return item_clusters[idx]

def get_cluster_items(cluster_id):
    return [item_idx for item_idx, cluster in enumerate(item_clusters) if cluster_id == cluster]

print(item_clusters[:50])
# print('-------', get_cluster_items(5))

def recommend(user_id, rec_num=30):
    user_interactions = interactions[interactions['user_id'] == user_id]
    print("num of interactions")
    print(np.shape(user_interactions))
    print(user_interactions)
    recommendations = []
    
    for _, item in user_interactions.iterrows():
        print(item['item_id'])
        cluster_id = get_item_cluster(item['item_id'])
        print("cluster id")
        print(cluster_id)
        print(get_item_cluster(2034642))
        print("=======================")
        cluster_items_idx = get_cluster_items(cluster_id)
        print(cluster_items_idx)
        cluster_items = [idx_to_item[x] for x in cluster_items_idx]
        
        if item['item_id'] in cluster_items_idx:
            print("YES")
        else:
            print("NOOOOOOOOOOO")
        print()
        if cluster_id not in similarities:
            similarities[cluster_id] = item_item_CF(cluster_items)

            local = {elem[0]: count for count, elem in enumerate(cluster_items)}
            local_indexing[cluster_id] = local
       
        similarity = similarities[cluster_id]
#         print(similarity[:5,:5])
        current_local_indexing = local_indexing[cluster_id]
        print(current_local_indexing)
        print("---------")
        print(item['item_id'])
        print(current_local_indexing[item['item_id']])
        x = similarity[current_local_indexing[item['item_id']]] 
        x = [(score, i) for i, score in enumerate(x)]
        x.sort()
        recommnedations.extend(x[:rec_num])
        
    recommendations.sort()
    return recommendations[:rec_num]
print("RESULT")
print(recommend(1731019))
        

/usr/local/lib/python3.6/dist-packages/scipy/cluster/vq.py:660: UserWarning: One of the clusters is empty. Re-run kmean with a different initialization.
  warnings.warn("One of the clusters is empty. "


[0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0
 1 1 0 1 1 1 0 1 1 0 1 0 1]
RESULT
num of interactions
(4, 4)
   user_id  item_id  interaction_type  created_at
3  1731019   265291                 1  1445613604
4  1731019  2397541                 1  1446822664
5  1731019  1860035                 1  1445613601
6  1731019  2397541                 3  1446822698
265291
cluster id
0
0
[0, 3, 4, 6, 7, 9, 10, 16, 17, 19, 23, 27, 28, 30, 32, 33, 36, 39, 43, 46, 48, 51, 56, 57, 60, 61, 64, 71, 73, 75, 82, 86, 89, 93, 94, 96, 97, 98, 105, 109, 113, 123, 128, 130, 131, 132, 133, 141, 145, 147, 150, 158, 163, 166, 169, 172, 174, 181, 182, 183, 186, 191, 194, 195, 201, 205, 209, 212, 213, 217, 218, 222, 225, 228, 229, 232, 237, 238, 239, 240, 243, 244, 248, 250, 268, 270, 272, 273, 279, 280, 282, 284, 286, 292, 294, 305, 319, 320, 321, 324, 328, 329, 348, 356, 357, 360, 366, 369, 370, 371, 373, 375, 382, 384, 385, 389, 394, 395, 400, 403, 406, 407, 408, 412, 414, 416, 417,

(44285, 9)
